# Modelos de regresión logística

Basado en el capítulo 4 de **Introducion to statistical learning with R**, de Gareth James, Daniea Witten, Trevor Hastie y Robert Tibshirani. Los ejemplos presentados en R en el libro fueron llevados a Python y completados.

El dataset que vamos a analizar contiene los registros del histórico de las personas a las que se les ha otorgado un crédito financiero y pudieron pagarlo o no.
La idea es poder predecir en un futuro, a partir de las variables independientes que aspirantes a un crédito tienen mas o menos riesgo de pagar.

Las variables de la hoja de datos son las siguientes:
- ID: El ID del cliente en el banco. Numérico, debe ser positivo y único
- PudoPagar: Indica si el cliente ha podido pagar el crédito sin problemas (1) o no (0). Variable categórica binaria
- Estudiante: Indica si el cliente es estudiante (Si) o no lo es (No). Variable categórica binaria
- Deuda: Indica la cantidad del crédito que aún adeuda el cliente. Deben ser valores numéricos positivos o nulos
- Ingresos: Indica el salario o demás ingresos de los que disponga. Deben ser valores numéricos positivos o nulos


## Entendimiento y preparación de los datos

Realice un análisis exploratorio de los datos verificando la calidad de los mismos. Utilice gráficos para poder identificar posibles problemas.



In [ ]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib as mpl
import matplotlib.pyplot as plt #gráficos
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import KFold, cross_val_score #protocolo de evaluación
from sklearn import metrics
from sklearn.preprocessing import scale 
import seaborn as sns

Cargamos los datos, que están en el mismo directorio que el notebook.

In [ ]:
data = pd.read_csv('02_creditos.csv', sep=';')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe(include="all")

In [ ]:
data.dtypes

La variable independiente student debe convertise en variable numérica. Vamos a utilizar el método factorize, que devuelve dos objetos: un array de labels y un array con los valores únicos que se encontraron y cuyas posiciones son los nuevos valores de los factores.

In [ ]:
data.student.factorize()

Solo nos interesamos en el primer array, que lo vamos a utilizar para remplazar los datos del dataframe

In [ ]:
data['default2'] = data.default.factorize()[0]
data['student2'] = data.student.factorize()[0]
data.head(3)

In [ ]:
### Baseline
ax = sns.countplot(x="default", data=data)
data.default[data.default=="No"].aggregate('count') / data.shape[0] * 100

Vamos a analizar los valores gráficamente.
Para que sean más lisibles y, dado que hay un gran desbalanceo entre los datos sin default y los datos con default, vamos a crear un nuevo Dataframe para poder visualizar los datos, en donde incluiremos todos los datos con default y una muestra con un porcentaje (15%) de los datos sin default.

In [ ]:
# Solo consideramos una muestra con una parte de los clientes que no tienen default
data_no = data[data.default2 == 0].sample(frac=0.15)

# Como son pocos, tomamos todos los datos de los clientes que si tienen default
data_yes = data[data.default2 == 1]
data_ = data_no.append(data_yes)
data.describe(include="all")

Creamos 3 plots:
- un scatterplot con las variables income y balance en los ejes y el color dado por la categoría del default
- un boxplot con las distribuciones de del income para las categorías con y sin default
- un boxplot con las distribuciones de del balance para las categorías con y sin default

Vamos a ignorar los warnings que no son importantes para lo que vamos a hacer

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig = plt.figure(figsize=(12,5))
gs = mpl.gridspec.GridSpec(1, 4)
ax1 = plt.subplot(gs[0,:-2])
ax2 = plt.subplot(gs[0,-2])
ax3 = plt.subplot(gs[0,-1])

# Plot de balance x income
ax1.scatter(data_[data_.default == 'Yes'].balance, data_[data_.default == 'Yes'].income, s=40, c='orange', marker='+',
            linewidths=1)
ax1.scatter(data_[data_.default == 'No'].balance, data_[data_.default == 'No'].income, s=40, marker='o', linewidths='1',
            edgecolors='lightblue', facecolors='white', alpha=.6)
ax1.set_ylim(ymin=0)
ax1.set_ylabel('Income')
ax1.set_xlim(xmin=-100)
ax1.set_xlabel('Balance')

# 2 Boxplots
c_palette = {'No':'lightblue', 'Yes':'orange'}
sns.boxplot('default', 'balance', data=data, orient='v', ax=ax2, palette=c_palette)
sns.boxplot('default', 'income', data=data, orient='v', ax=ax3, palette=c_palette)
gs.tight_layout(plt.gcf())

Podemos ver que la variable *balance* es bastante discriminante con respecto a la variable dependiente.

## Modelamiento: regresión logística vs. regresión lineal

### Variable independiente: balance

Vamos a analizar la relación estrecha entre la variable independiente *balance* con la dependiente *default* para ilustrar los problemas que tendría el utilizar la regresión linear para tareas de clasificación, al mismo tiempo que comparamos ese modelo con el de la regresión logística.

Primero preparamos los datos en dos variables, una matriz de una sola columna para la variable predictiva y un array con los valores de la variable objetivo.

In [ ]:
X = data.balance.values.reshape(-1,1) #Una columna, con las filas que se necesiten
X.shape

In [ ]:
y = data.default2
y.shape

Con el propósito de poder visualizar el resultado de los modelos de clasificación, creamos un array con todos los valores enteros entre el más pequeño y el más grande de la variable indepediente balance y predecimos el valor de la variable dependiente de estos valores.

In [ ]:
X_plot = np.arange(data.balance.min(), data.balance.max()).reshape(-1,1)
print("El rango de la variable balance es [{}, {}]".format(data.balance.min(), data.balance.max()))

In [ ]:
X_plot[0:5]

Creamos el modelo de LogReg, predecimos las probabilidades para todos los valores en el rango de la variable predictiva.

In [ ]:
clf = LogisticRegression()
clf.fit(X,y)
prob = clf.predict_proba(X_plot)
prob

Ploteamos el resultado de la regresión lineal en el plot izquierdo y de la regresión logística en el plot de la derecha.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))

# Izquierda: plot de regresión lineal: se le dan los datos del eje x, del eje y, el orden polinómico
# el nivel de confianza para un intervalo de confianza (ci), parámetros para el plot de scatter,
# y parámetros para el plot lineal
sns.regplot(data.balance, data.default2, order=1, ci=None,
            scatter_kws={'color':'orange'},
            line_kws={'color':'lightblue', 'lw':2}, ax=ax1)

# Derecha: plot de regresión logística: un scatterplot con los valores del balance y del default (0/1),
# y un plot con las probabilidades resultado de la regresión logística.
ax2.scatter(X, y, color='orange')
ax2.plot(X_plot, prob[:,1], color='lightblue')

# Se agregan las lineas de los valores 0 y 1 y los títulos de los ejes
for ax in fig.axes:
    ax.hlines(1, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1)
    ax.hlines(0, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1)
    ax.set_ylabel('Probability of default')
    ax.set_xlabel('Balance')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.])
    ax.set_xlim(xmin=-100)

Los resultados de la regresión logística con la variable *balance* nos dan:

In [ ]:
clf = LogisticRegression()

X = data.balance.values.reshape(-1,1)
clf.fit(X,y)
print(clf)
print('clases: ',clf.classes_)
print('coeficientes: ',clf.coef_)
print('intercepto :', clf.intercept_)
y_pred = clf.predict(X)
cm= metrics.confusion_matrix(y, y_pred)
print(cm)
print("Exactitud: ", metrics.accuracy_score(y, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y, y_pred))

Al igual que con la regresión lineal, scikit-learn no facilita los valores-p de la significancia de los coeficientes, por lo que utilizaremos **statsmodels**

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

X = sm.add_constant(data.balance)
est = sm.Logit(y.ravel(), X).fit()
est.summary2().tables[1]

Encontramos que el coeficiente de *balance* positvo, y tiene un valor-p inferior a un nivel de significancia del 5% o del 1%, por lo que podemos decir que la relación entre balance y default es significativa.

### Variable independiente: student

In [ ]:
X = sm.add_constant(data.student2)
est = sm.Logit(y.ravel(), X).fit()
est.summary2().tables[1]

Vemos que la variable *student* tiene un coeficiente positivo, lo que significa que los estudiantes son más propensos a tener default que los no estudiantes, y su valor-p es significativo.

### Variables independientes: balance, income y student (regresión logística múltiple)

In [ ]:
X = sm.add_constant(data[['balance', 'income', 'student2']])
est = smf.Logit(y, X).fit()
est.summary2().tables[1]

En el modelo de regresión logística múltiple nos da que income no es significativa (valor-p del 71%), aunque las otras dos si.
En este modelo múltiple, el coeficiente de *student* nos dice que los estudiantes son menos propensos al default que los no estudiantes (coeficiente negativo). 
En la regresión logística que solo consideraba *student* encontrabamos que la relación  era contraria.

Esto se puede explicar por la relación entre *student* y *balance* (no incluimos *income*, al no ser significativa). 
Vamos a analizar por separado la subpoblación estudiante y la no estudiante para tratar de entender esta relación entre *student* y *default* dado la presencia de *balance*.

In [ ]:
# Dividimos los datos entre los estudiantes y los no estudiantes.
X_stud = data[data.student == 'Yes'].balance.values.reshape(data[data.student == 'Yes'].balance.size,1) 
y_stud = data[data.student == 'Yes'].default2
X_nostud = data[data.student == 'No'].balance.values.reshape(data[data.student == 'No'].balance.size,1) 
y_nostud = data[data.student == 'No'].default2

Aprendemos los dos modelos de regresión para los dos datasets, solo considerando *balance* como variable independiente.
Al igual que anteriormente, evaluamos para el rango de valores posibles de la variable *balance*.

In [ ]:
#Aprendizaje de los modelos
logreg_stud   = LogisticRegression()
logreg_nostud = LogisticRegression()
logreg_stud.fit  (  X_stud, y_stud  )
logreg_nostud.fit(X_nostud, y_nostud)

# Evaluación de los modelos
X_plot = np.arange(data.balance.min(), data.balance.max()).reshape(-1,1)
prob_stud = logreg_stud.predict_proba(X_plot)
prob_nostud = logreg_nostud.predict_proba(X_plot)

In [ ]:
y_pred_stud = clf.predict(X_stud)
cm= metrics.confusion_matrix(y_stud, y_pred_stud)
print(cm)

data.default[data.default=="No"].aggregate('count') / data.shape[0] * 100
print("Baseline: ", (len(y_stud)-np.count_nonzero(y_stud)) / len(y_stud))
print("Exactitud: ", metrics.accuracy_score(y_stud, y_pred_stud))
print("Kappa    : ", metrics.cohen_kappa_score(y_stud, y_pred_stud))

In [ ]:
y_pred_nostud = clf.predict(X_nostud)
cm= metrics.confusion_matrix(y_nostud, y_pred_nostud)
print(cm)
print("Baseline: ", (len(y_nostud)-np.count_nonzero(y_nostud)) / len(y_nostud))
print("Exactitud: ", metrics.accuracy_score(y_nostud, y_pred_nostud))
print("Kappa    : ", metrics.cohen_kappa_score(y_nostud, y_pred_nostud))

In [ ]:
data.groupby(['student','default']).size().unstack('student')

Creamos dos plots para poder entender la relación entre *student* y *balance*

In [ ]:
# creating plot
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))

# Left plot
ax1.plot(X_plot, pd.DataFrame(prob_stud)[1], color='orange', label='Estudiantes')
ax1.plot(X_plot, pd.DataFrame(prob_nostud)[1], color='lightblue', label='No estudiantes')
ax1.hlines(127/2817, colors='orange', label='Estudiantes, global',
           xmin=ax1.xaxis.get_data_interval()[0],
           xmax=ax1.xaxis.get_data_interval()[1], linestyles='dashed')
ax1.hlines(206/6850, colors='lightblue', label='No estudiantes, global',
           xmin=ax1.xaxis.get_data_interval()[0],
           xmax=ax1.xaxis.get_data_interval()[1], linestyles='dashed')
ax1.set_ylabel('Default')
ax1.set_xlabel('Balance')
ax1.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.])
ax1.set_xlim(450,2500)
ax1.legend(loc=2)

# Right plot
sns.boxplot('student', 'balance', data=data, orient='v', ax=ax2,  palette=c_palette);

Vemos a la derecha que los estudiantes tienen valores mayores de *balance* que los no estudiantes. Como las distribuciones de *balance* para estudiantes y no estudiantes son diferentes, podemos decir que hay una correlación.

En el gráfico de la izquierda vemos las líneas horizontales punteadas de probabilidad globales de *default* (sin considerar el balance), con la de los estudiantes por encima de la de los no estudiantes.
Sin embargo, cuando consideramos la correlación de *student* con *balance*, encontramos que a mayores valores de balance, la probabilidad de default es mayor para los no estudiantes que para los estudiantes.

Este caso ilustra las sutilezas asociadas a las regresiones realizadas con una sola variable, cuando otros predictores pueden ser también relevantes. También muestra los posibles efectos de las correlaciones entre predictores. Este fenómeno se llama **"confounding"**.